# Showdown I/O
Showdown I/O is a Python-Selenium-based interface for autonomous battle monitoring and play on Pokémon Showdown. With it, you can automatically participate in or just collect information about battles happening on the world's most popular Pokémon battle simulator. 

Supported actions include:
- Logging in as a specified user
- Collecting maximally comprehensive representations of any battle's current state and history
- Starting and ending battles on a selected ladder or with a specific user
- Playing through battles by submitting decisions and monitoring their consequences for the battle state

## Setup
### Dependencies
how to make more colab friendly? Maybe I can animate stuff with screenshots? The screenshots don't capture whole screen anyway. Maybe there's a way to capture whole window? Render whole HTML in my browser, perhaps? Hm, would definitely require detailed javascript/python action. Worthwhile? Probably not...

In [0]:
# the dummy browser
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.support import ui
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import os

# generic site url
siteurl = 'https://play.pokemonshowdown.com/'

# paths that showdown-io will interact with
startbattle = '//*[@id="room-"]/div/div[1]/div[2]/div[1]/form/p[3]/button'
enterusername = '/html/body/div[4]/div/form/p[1]/label/input'
findbattle = "//button[@name='search']"
enterpassword = '/html/body/div[4]/div/form/p[4]/label/input'
battlehistory = '//div[@class="battle-history"]'

# path to web driver (need to generalize!)
driverpath = os.path.join(os.getcwd(), 'geckodriver')

### The ShowdownIO Class
Need to have protocol if username/password is invalid, and if new account must be registered.

Need startbattle to either work when a battle already exists, or explicitly not work. Can resolve by always clicking "Home" before starting battle maybe.

Battlehistory is properly a sequence of gamestates, not lines of battle log. One new entry at start of every turn, maintained internally.

Gamestate function thus needs to return sequence[-1] of this battle history.

What does GameState represent and how?
An entry for everything relevant to the damage calculator, and one for every pokemon on my or enemy team. Also need extra stuff counting remaining duration of stuff (do I??), delayed effects (I do), etc. Also have to track PP.
Later on, need to be able to fill in missings with predicted values.



In [0]:
class ShowdownIO(object):
    def __init__(self, username, password):
        "Initiate a bot with specified username and password."
        # start browser and open site
        self.driver = webdriver.Firefox(FirefoxProfile(), executable_path=driverpath)
        self.wait = ui.WebDriverWait(self.driver, 10)
        self.driver.get(siteurl)      
        
        # set username and password
        self.wait.until(lambda _: self.xpath(findbattle).text == 'Battle!\nFind a random opponent')
        self.xpath(findbattle).click()
        self.xpath(enterusername).send_keys(username)
        self.xpath(enterusername).send_keys(Keys.RETURN)
        self.wait.until(lambda _: self.xpath(enterpassword) is not None)
        self.xpath(enterpassword).send_keys(password)
        self.xpath(enterpassword).send_keys(Keys.RETURN)
        
        # begin battlehistory representation
        
    def startbattle(self):
        "Start a random battle. Fails if already in a battle."
        self.xpath(findbattle).click()
        self.wait.until(lambda _: self.xpath(battlehistory) is not None)
    
    def battlehistory(self):
        if self.xpath(battlehistory) is None:
            return None
        return [each.text for each in self.driver.find_elements_by_xpath(battlehistory)]
    
    def team(self):
        members = []
        for each in battlebot.driver.find_elements_by_xpath('//button[@name="chooseSwitch" or @name="chooseDisabled"]'):
            ActionChains(battlebot.driver).move_to_element(each).perform()
            members.append(battlebot.driver.find_element_by_xpath('//div[@class="tooltip"]').text)
        members = [self.parsemember(member) for member in members]
        return members
    
    def parsemember(self, member):
        lines = member.split('\n')
        parsed = {}

        # name/level line
        namelevel = lines[0]
        parsed['name'] = namelevel[:namelevel.rfind(' ')]
        parsed['level'] = int(namelevel[namelevel.rfind('L')+1:])

        # hp line
        if 'fainted' in lines[1]:
            parsed['hp'] = 'fainted'
        else:
            hpline = lines[1].split(' ')
            parsed['hp'] = {'percentage': float(hpline[1][:-1]),
                  'max': float(hpline[2][hpline[2].find('/')+1:-1])}

        # ability/item line
        abilityitem = lines[2].split(' / ')
        parsed['ability'], parsed['item'] = abilityitem[0][9:], abilityitem[1][6:]

        # stats
        for each in lines[3].split(' / '):
            parsed[each[:3]] = int(each[4:])

        # moves
        parsed['moves'] = [each[2:] for each in lines[4:]]

        return parsed
        
    def xpath(self, xpath):
        return self.driver.find_element_by_xpath(xpath)

In [0]:
battlebot = Battlebot('magicalnumber7', 'password') # you should make your own account to test this, though later we'll share a testing account

In [0]:
battlebot.startbattle()

ElementNotInteractableException: Message: 


In [0]:
battlebot.battlehistory()

['Battle started between magicalnumber7 and paraent!',
 'Go! Forretress!',
 'paraent sent out Keldeo!',
 'The opposing Keldeo used Calm Mind!',
 "The opposing Keldeo's Special Attack rose!",
 "The opposing Keldeo's Special Defense rose!",
 'Forretress used Gyro Ball!',
 "It's not very effective...",
 '(The opposing Keldeo lost 14% of its health!)',
 'Forretress, come back!',
 'Go! Illumise!',
 'The opposing Keldeo used Icy Wind!',
 'Illumise avoided the attack!',
 'Illumise used Defog!',
 "The opposing Keldeo's evasiveness fell!",
 'The opposing Keldeo used Scald!',
 '(Illumise lost 54.9% of its health!)',
 'Illumise restored a little HP using its Leftovers!',
 'Illumise used Defog!',
 "The opposing Keldeo's evasiveness fell!",
 'The opposing Keldeo used Scald!',
 '(Illumise lost 51.2% of its health!)',
 'Illumise fainted!',
 'Go! Giratina (Giratina-Origin)!',
 'The opposing Keldeo used Calm Mind!',
 "The opposing Keldeo's Special Attack rose!",
 "The opposing Keldeo's Special Defense 

In [0]:
(battlebot.team())

[{'Atk': 218,
  'Def': 188,
  'SpA': 109,
  'SpD': 188,
  'Spe': 174,
  'ability': 'Levitate',
  'hp': {'max': 339.0, 'percentage': 100.0},
  'item': 'Griseous Orb',
  'level': 73,
  'moves': ['Dragon Tail', 'Shadow Ball', 'Draco Meteor', 'Earthquake'],
  'name': 'Giratina (Giratina-Origin)'},
 {'Atk': 83,
  'Def': 174,
  'SpA': 171,
  'SpD': 191,
  'Spe': 191,
  'ability': 'Prankster',
  'hp': 'fainted',
  'item': 'Leftovers',
  'level': 84,
  'moves': ['Defog', 'Roost', 'Bug Buzz', 'Thunder Wave'],
  'name': 'Illumise'},
 {'Atk': 188,
  'Def': 267,
  'SpA': 140,
  'SpD': 140,
  'Spe': 68,
  'ability': 'Sturdy',
  'hp': {'max': 248.0, 'percentage': 100.0},
  'item': 'Leftovers',
  'level': 79,
  'moves': ['Gyro Ball', 'Volt Switch', 'Stealth Rock', 'Spikes'],
  'name': 'Forretress'},
 {'Atk': 248,
  'Def': 185,
  'SpA': 248,
  'SpD': 185,
  'Spe': 193,
  'ability': 'Pressure',
  'hp': {'max': 323.0, 'percentage': 100.0},
  'item': 'Leftovers',
  'level': 78,
  'moves': ['Outrage', 'Ro

In [0]:
a[0]

'Giratina (Giratina-Origin) L73\nHP: 100.0% (339/339)\nAbility: Levitate / Item: Griseous Orb\nAtk 218 / Def 188 / SpA 218 / SpD 188 / Spe 174\n• Dragon Tail\n• Shadow Ball\n• Draco Meteor\n• Earthquake'

In [0]:
parsemember(a[0])

{'Atk': 218,
 'Def': 188,
 'SpA': 218,
 'SpD': 188,
 'Spe': 174,
 'ability': 'Levitate',
 'hp': {'max': 339.0, 'percentage': 100.0},
 'item': 'Griseous Orb',
 'level': 73,
 'moves': ['Dragon Tail', 'Shadow Ball', 'Draco Meteor', 'Earthquake'],
 'name': 'Giratina (Giratina-Origin)'}